In [1]:
from dotenv import load_dotenv

load_dotenv()

True

### Input & Output State

In [4]:
from langchain_openai import ChatOpenAI
from langgraph.graph import END, START, StateGraph

model = ChatOpenAI(model = "gpt-4o-mini")

In [5]:
from typing import TypedDict


class ChatMessages(TypedDict):
    question: str
    answer: str
    llm_calls: int

In [6]:
def call_model(state: ChatMessages):
    question = state["question"]
    llm_calls = state.get("llm_calls", 0)
    state["llm_calls"] = llm_calls + 1
    print("LLM_CALLS:", state["llm_calls"])
    response = model.invoke(input=question)
    state["answer"] = response.content
    return state

In [7]:
workflow = StateGraph(ChatMessages)

workflow.add_edge(START, "agent")
workflow.add_node("agent", call_model)
workflow.add_edge("agent", END)

graph = workflow.compile()

In [8]:
# In this case we are not interested in the overall output but rather just the answer and the llm_calls was just an example to
#show that we dont want it
graph.invoke(input={"question": "Whats the highest mountain in the world?"})

LLM_CALLS: 1


{'question': 'Whats the highest mountain in the world?',
 'answer': 'The highest mountain in the world is Mount Everest, which stands at an elevation of 8,848.86 meters (29,031.7 feet) above sea level. It is located in the Himalayas on the border between Nepal and the Tibet Autonomous Region of China.',
 'llm_calls': 1}

In [9]:
from langgraph.graph import StateGraph, START, END
from typing_extensions import TypedDict


class InputState(TypedDict):
    question: str


class PrivateState(TypedDict):
    llm_calls: int


class OutputState(TypedDict):
    answer: str


class OverallState(InputState, PrivateState, OutputState):
    pass

In [10]:
workflow = StateGraph(OverallState, input=InputState, output=OutputState)

workflow.add_edge(START, "agent")
workflow.add_node("agent", call_model)
workflow.add_edge("agent", END)

graph = workflow.compile()

In [11]:
# This way we got rid of the extra keys that we dont need
graph.invoke({"question": "Whats the highest mountain in the world?"})

LLM_CALLS: 1


{'answer': 'The highest mountain in the world is Mount Everest, which stands at an elevation of 8,848.86 meters (29,031.7 feet) above sea level. It is located in the Himalayas on the border between Nepal and the Tibet Autonomous Region of China.'}

### Add runtime configuration

In [ ]:
from langgraph.graph import StateGraph, START, END
from langchain_core.runnables.config import RunnableConfig
from langchain.schema import SystemMessage, HumanMessage


def call_model(state: OverallState, config: RunnableConfig):
    language = config["configurable"].get("language", "English") #Any config must be ran inside "configuration" key
    system_message_content = "Respond in {}".format(language)
    system_message = SystemMessage(content=system_message_content)
    messages = [system_message, HumanMessage(content=state["question"])]
    response = model.invoke(messages)
    return {"answer": response}

In [ ]:
workflow = StateGraph(ChatMessages)

workflow.add_edge(START, "agent")
workflow.add_node("agent", call_model)
workflow.add_edge("agent", END)

graph = workflow.compile()

In [ ]:
config = {"configurable": {"language": "Spanish"}}
graph.invoke({"question": "What's the highest mountain in the world?"}, config=config)

In [ ]:
config = {"configurable": {"language": "German"}}
graph.invoke({"question": "What's the highest mountain in the world?"}, config=config)